<a href="https://colab.research.google.com/github/hazrakeruboO/DS-Colabs/blob/main/Copy_of_code_example_of_a_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code example of text generator
Adapted from: https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

In [ ]:
# Importing libraries needed for processing data
import numpy as np
import string as st
import re
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

# Libraries needed for model building
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

In [ ]:
# Initializing text, "Corpus"
text = "Students at AIMS have the best time in the world. For ten month, the get to sit and engage with the best minds and challenge their Mathematical thinking. They learn, work and best of all play."
text

'Students at AIMS have the best time in the world. For ten month, the get to sit and engage with the best minds and challenge their Mathematical thinking. They learn, work and best of all play.'

In [ ]:
# Removing punctuations
text = re.sub(r'[^\w\s]', '', text) 

# Splitting the text
tokens = text.lower().split()
print(tokens)
print(len(tokens))

['students', 'at', 'aims', 'have', 'the', 'best', 'time', 'in', 'the', 'world', 'for', 'ten', 'month', 'the', 'get', 'to', 'sit', 'and', 'engage', 'with', 'the', 'best', 'minds', 'and', 'challenge', 'their', 'mathematical', 'thinking', 'they', 'learn', 'work', 'and', 'best', 'of', 'all', 'play']
36


In [ ]:
# Splitting into equal sequences
length = 5
lines = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	lines.append(line)
print('Total Sequences: %d' % len(lines))

Total Sequences: 31


In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences[0]

[27, 25, 24, 23, 1]

In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# separate into input and output
sequences = np.array(sequences)
#X, y = sequences[:,:-1], sequences[:,-1]
X, y = sequences,sequences[:,-1]
y = np_utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
# define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=seq_length))
model.add(LSTM(10, return_sequences=True))
model.add(LSTM(10))
model.add(Dense(10, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 5)              145       
_________________________________________________________________
lstm (LSTM)                  (None, 5, 10)             640       
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_1 (Dense)              (None, 29)                319       
Total params: 2,054
Trainable params: 2,054
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, 1, input_length=seq_length))
# model.add(LSTM(3, return_sequences=True))
# model.add(LSTM(3))
# #model.add(Dense(3, activation='relu'))
# model.add(Dense(vocab_size, activation='softmax'))
# # compile model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit model
model.fit(X, y, batch_size=1, epochs=200, verbose=1)

Epoch 1/200
31/31 [==============================] - 21s 5ms/step - loss: 3.3686 - accuracy: 0.0233
Epoch 2/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3619 - accuracy: 0.0404
Epoch 3/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3541 - accuracy: 0.0583
Epoch 4/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3448 - accuracy: 0.1882
Epoch 5/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3354 - accuracy: 0.0802
Epoch 6/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3388 - accuracy: 0.0555
Epoch 7/200
31/31 [==============================] - 0s 4ms/step - loss: 3.3295 - accuracy: 0.0945
Epoch 8/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3229 - accuracy: 0.1370
Epoch 9/200
31/31 [==============================] - 0s 5ms/step - loss: 3.2914 - accuracy: 0.1783
Epoch 10/200
31/31 [==============================] - 0s 5ms/step - loss: 3.3194 - accuracy: 0.0394
Epoch 11

In [ ]:
# save the model to file
model.save('model.h5')
# save the tokenizer
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load the model
from keras.models import load_model
model = load_model('model.h5')

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
# select a seed text
seed_text = lines[np.random.randint(0,len(lines))]
print(seed_text + '\n')

with the best minds and



In [ ]:
seq_length = len(lines[0].split()) - 1
seq_length

4

In [ ]:
# encode the text as integer
encoded = tokenizer.texts_to_sequences([seed_text])[0]

from keras.preprocessing.sequence import pad_sequences
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
encoded

array([[ 1,  3, 15,  2]], dtype=int32)

In [ ]:
# predict probabilities for each word
yhat = np.argmax(model.predict(encoded))
yhat

18

In [ ]:
# map predicted word index to word
out_word = ''
for word, index in tokenizer.word_index.items():
  if index == yhat:
    out_word = word
    break

print(out_word)

mathematical
